In [1]:
from tensorflow.keras.datasets import cifar10

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train.shape

(50000, 32, 32, 3)

In [4]:
y_train.shape

(50000, 1)

In [5]:
from tensorflow.keras.utils import normalize

x_train = normalize(x_train)
x_test = normalize(x_test)

In [6]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0.0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
        brightness_range=None,  
        shear_range=0.0,  # set range for random shear
        zoom_range=0.0,  # set range for random zoom
        channel_shift_range=0.0,  # set range for random channel shifts
        fill_mode='nearest',  # set mode for filling points outside the input boundaries
        cval=0.0,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale=None,  # set rescaling factor (applied before any other transformation)
        preprocessing_function=None,  # set function that will be applied on each input
        data_format=None,  # image data format, either "channels_first" or "channels_last"
        validation_split=0.0  # fraction of images reserved for validation (strictly between 0 and 1)
)

datagen.fit(x_train)

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import ZeroPadding2D, Conv2D, MaxPooling2D, Dropout, Activation, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD

In [9]:
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler

In [10]:
import datetime
#NAME = 'cifar10-8blocks-elu-lr-schedule-l2-regulizer-horizontal-flipping-{}'.format(datetime.datetime.now())
NAME = 'cifar10-7blocks-my_activation-lr-schedule-l2-regulizer-horizontal-flipping-{}'.format(datetime.datetime.now())
tensorboard = TensorBoard(log_dir='./logs/{}'.format(NAME),
                          #histogram_freq=2,
                          #write_graph=True,
                          #write_grads=True,
                          #write_images=True,
                         )

In [11]:
def lr_schedule(epoch, lr):
    if epoch < 70:
        return 0.01
    elif epoch < 170:
        return 0.005
    elif epoch < 270:
        return 0.0005
    else:
        return 0.00005
schedule = LearningRateScheduler(lr_schedule)

In [12]:
import tensorflow as tf
def my_activation(x):
    return tf.where(x > 0.0,
                    tf.log(tf.maximum(x, 0.0) + 1.0),
                    -tf.log(-tf.minimum(x, 0.0) + 1.0))

In [ ]:
model = Sequential([
    #Block 1
    ZeroPadding2D(input_shape=(32, 32, 3)),
    Conv2D(384, 3, kernel_regularizer=l2(l=0.0005)), # 32
    Activation('relu'),
    MaxPooling2D(), # 16
    # Block 2
    Conv2D(384, 1, kernel_regularizer=l2(l=0.0005)), # 16
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(384, 2, kernel_regularizer=l2(l=0.0005)), # 17
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(640, 2, kernel_regularizer=l2(l=0.0005)), # 18
    Activation('relu'),
    MaxPooling2D(), # 9
    Dropout(0.1),
    # Block 3
    Conv2D(640, 1, kernel_regularizer=l2(l=0.0005)), # 9
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(768, 2, kernel_regularizer=l2(l=0.0005)), # 10
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(768, 2, kernel_regularizer=l2(l=0.0005)), # 11
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(768, 2, kernel_regularizer=l2(l=0.0005)), # 12
    Activation('relu'),
    MaxPooling2D(), # 6
    Dropout(0.2),
    # Block 4
    Conv2D(768, 1, kernel_regularizer=l2(l=0.0005)), # 6
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(896, 2, kernel_regularizer=l2(l=0.0005)), # 7
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(896, 2, kernel_regularizer=l2(l=0.0005)), # 8
    Activation('relu'),
    MaxPooling2D(), # 4
    Dropout(0.3),
    # Block 5
    Conv2D(896, 1, kernel_regularizer=l2(l=0.0005)), # 4
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(1024, 2, kernel_regularizer=l2(l=0.0005)), # 5
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(1024, 2, kernel_regularizer=l2(l=0.0005)), # 6
    Activation('relu'),
    MaxPooling2D(), # 3
    Dropout(0.4),
    # Block 6
    Conv2D(1024, 1, kernel_regularizer=l2(l=0.0005)), # 3
    Activation('relu'),
    Conv2D(1152, 2, kernel_regularizer=l2(l=0.0005)), # 2
    Activation('relu'),
    MaxPooling2D(), # 1
    Dropout(0.5),
    # Block 7
    Conv2D(1152, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation('relu'),
    Dropout(0.5),
    # Block 8
    Conv2D(10, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation('sigmoid'),
    Flatten(),
])

In [13]:
model = Sequential([
    # Block 1
    ZeroPadding2D(padding=(4, 4), input_shape=(32, 32, 3)),
    Conv2D(192, 5, kernel_regularizer=l2(l=0.0005)), # 36
    Activation(my_activation),
    MaxPooling2D(), # 18
    # Block 2
    Conv2D(192, 1, kernel_regularizer=l2(l=0.0005)), # 18
    Activation(my_activation),
    ZeroPadding2D(),
    Conv2D(240, 3, kernel_regularizer=l2(l=0.0005)), # 18
    Activation(my_activation),
    MaxPooling2D(), # 9
    Dropout(0.1),
    # Block 3
    Conv2D(240, 1, kernel_regularizer=l2(l=0.0005)), # 9
    Activation(my_activation),
    ZeroPadding2D(),
    Conv2D(260, 2, kernel_regularizer=l2(l=0.0005)), # 10
    Activation(my_activation),
    MaxPooling2D(), # 5
    Dropout(0.2),
    # Block 4
    Conv2D(260, 1, kernel_regularizer=l2(l=0.0005)), # 5
    Activation(my_activation),
    ZeroPadding2D(),
    Conv2D(280, 2, kernel_regularizer=l2(l=0.0005)), # 6
    Activation(my_activation),
    MaxPooling2D(), # 3
    Dropout(0.3),
    # Block 5
    Conv2D(280, 1, kernel_regularizer=l2(l=0.0005)), # 3
    Activation(my_activation),
    Conv2D(300, 2), # 2
    Activation(my_activation),
    MaxPooling2D(), # 1
    Dropout(0.4),
    # Block 6
    Conv2D(300, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation(my_activation),
    Dropout(0.5),
    # Block 7
    Conv2D(10, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation('sigmoid'),
    Flatten()
])

In [14]:
from tensorflow.keras.metrics import top_k_categorical_accuracy

In [15]:
model.compile(#'adam',
              SGD(lr=0.01, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy', top_k_categorical_accuracy])

In [ ]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                           validation_data=[x_test, y_test],
                           epochs=330,
                           callbacks=[tensorboard, schedule]
                          )

Epoch 1/330
500/500 [==============================] - 47s 94ms/step - loss: 3.1157 - acc: 0.1917 - top_k_categorical_accuracy: 0.6991 - val_loss: 2.9203 - val_acc: 0.2700 - val_top_k_categorical_accuracy: 0.8104
Epoch 2/330
500/500 [==============================] - 45s 90ms/step - loss: 2.8198 - acc: 0.2827 - top_k_categorical_accuracy: 0.8192 - val_loss: 2.6224 - val_acc: 0.3270 - val_top_k_categorical_accuracy: 0.8724
Epoch 3/330
500/500 [==============================] - 45s 90ms/step - loss: 2.5985 - acc: 0.3457 - top_k_categorical_accuracy: 0.8574 - val_loss: 2.4535 - val_acc: 0.3750 - val_top_k_categorical_accuracy: 0.8825
Epoch 4/330
500/500 [==============================] - 45s 90ms/step - loss: 2.4364 - acc: 0.3871 - top_k_categorical_accuracy: 0.8776 - val_loss: 2.2918 - val_acc: 0.4269 - val_top_k_categorical_accuracy: 0.8985
Epoch 5/330
500/500 [==============================] - 45s 90ms/step - loss: 2.2979 - acc: 0.4166 - top_k_categorical_accuracy: 0.8928 - val_loss: 2

500/500 [==============================] - 45s 90ms/step - loss: 1.2203 - acc: 0.6620 - top_k_categorical_accuracy: 0.9660 - val_loss: 1.2037 - val_acc: 0.6612 - val_top_k_categorical_accuracy: 0.9656
Epoch 40/330
500/500 [==============================] - 45s 90ms/step - loss: 1.2201 - acc: 0.6593 - top_k_categorical_accuracy: 0.9652 - val_loss: 1.1933 - val_acc: 0.6651 - val_top_k_categorical_accuracy: 0.9673
Epoch 41/330
500/500 [==============================] - 45s 90ms/step - loss: 1.2114 - acc: 0.6593 - top_k_categorical_accuracy: 0.9661 - val_loss: 1.2163 - val_acc: 0.6595 - val_top_k_categorical_accuracy: 0.9627
Epoch 42/330
500/500 [==============================] - 45s 90ms/step - loss: 1.2064 - acc: 0.6644 - top_k_categorical_accuracy: 0.9658 - val_loss: 1.2090 - val_acc: 0.6614 - val_top_k_categorical_accuracy: 0.9644
Epoch 43/330
500/500 [==============================] - 45s 90ms/step - loss: 1.1964 - acc: 0.6692 - top_k_categorical_accuracy: 0.9663 - val_loss: 1.2027 - 

500/500 [==============================] - 45s 90ms/step - loss: 1.0421 - acc: 0.7267 - top_k_categorical_accuracy: 0.9771 - val_loss: 1.1194 - val_acc: 0.6987 - val_top_k_categorical_accuracy: 0.9723
Epoch 78/330
500/500 [==============================] - 45s 90ms/step - loss: 1.0436 - acc: 0.7230 - top_k_categorical_accuracy: 0.9766 - val_loss: 1.1441 - val_acc: 0.6889 - val_top_k_categorical_accuracy: 0.9708
Epoch 79/330
500/500 [==============================] - 45s 90ms/step - loss: 1.0411 - acc: 0.7255 - top_k_categorical_accuracy: 0.9770 - val_loss: 1.1090 - val_acc: 0.7025 - val_top_k_categorical_accuracy: 0.9716
Epoch 80/330
500/500 [==============================] - 45s 90ms/step - loss: 1.0372 - acc: 0.7261 - top_k_categorical_accuracy: 0.9778 - val_loss: 1.1029 - val_acc: 0.7035 - val_top_k_categorical_accuracy: 0.9733
Epoch 81/330
500/500 [==============================] - 45s 90ms/step - loss: 1.0296 - acc: 0.7286 - top_k_categorical_accuracy: 0.9775 - val_loss: 1.1300 - 